In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Pipelines: Lightweight Python基于函数的组件，以及组件I/O

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/lightweight_functions_component_io_kfp.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在Colab中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fnotebook_template.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab Enterprise中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/pipelines/lightweight_functions_component_io_kfp.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在GitHub上查看
    </a>
  </td>
  <td style="text-align: center">
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/pipelines/lightweight_functions_component_io_kfp.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在Vertex AI Workbench中打开
     </a>
   </td>
</table>
<br/><br/><br/>

## 概述

本笔记本展示了如何使用[Kubeflow Pipelines（KFP）SDK](https://www.kubeflow.org/docs/components/pipelines/)来构建[Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines)，这些管道使用基于轻量级Python函数的组件，同时支持使用KFP SDK进行组件I/O。

了解更多关于[Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction)的信息。

### 目标

在本教程中，您将学习如何使用KFP SDK构建基于轻量级Python函数的组件，然后学习如何使用Vertex AI Pipelines执行流水线。

本教程使用以下谷歌云ML服务：

- Vertex AI Pipelines

其中执行的步骤包括：

- 构建基于Python函数的KFP组件。
- 构建一个KFP流水线。
- 在组件之间传递*Artifacts*和*parameters*，既通过路径引用，也通过值。
- 使用kfp.dsl.importer方法。
- 编译KFP流水线。
- 使用Vertex AI Pipelines执行KFP流水线。

### KFP 基于 Python 函数的组件

Kubeflow 管道组件是一个自包含的代码集，用于执行机器学习工作流程中的一个步骤。管道组件由以下组成：

* 组件代码，实现执行机器学习工作流程中步骤所需逻辑的代码。
* 组件规范，定义以下内容：
  * 组件的元数据，名称和描述。
  * 组件的接口，组件的输入和输出。
* 组件的实现，Docker 容器映像用于运行，如何将输入传递给您的组件代码，以及如何获取组件的输出。

轻量级的基于 Python 函数的组件使得通过将组件代码构建为 Python 函数并为您生成组件规范，进而更容易快速迭代。本笔记本展示了如何为在 [Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines) 中使用创建基于 Python 函数的组件。

基于 Python 函数的组件使用 Kubeflow Pipelines SDK 处理将输入传递给您的组件以及将您的函数输出返回到管道中的复杂性。

基于 Python 函数的组件支持两种输入/输出类别：*工件* 和 *参数*。

* 参数通过值传递给您的组件，通常包含 `int`、`float`、`bool` 或小型`string` 值。
* 工件作为对路径的*引用*传递给您的组件，您可以写入文件或子目录结构。除了工件的数据，您还可以读取和写入工件的元数据。这让您记录工件的任意键值对，例如训练模型的准确率，并在下游组件中使用元数据 - 例如，您可以使用元数据来决定是否模型足够准确以用于预测部署。

### 成本

本教程使用 Google Cloud 中的可计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 定价](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据预期使用量生成成本估算。

开始吧

安装Python的Vertex AI SDK和其他必需的包。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-storage \
                                 kfp \
                                 "numpy<2" \
                                 google-cloud-pipeline-components

重新启动运行时（仅适用于Colab）
在Google Colab上进行环境验证。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### 验证您的笔记本环境（仅限Colab）
在谷歌Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置谷歌云项目信息
了解更多关于如何设置项目和开发环境。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

- *{Note to notebook author: 对于任何需要唯一的用户提供的字符串（如存储桶名称或模型 ID），请在末尾附加“-unique”以便进行适当的测试}*

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶尚不存在：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

#### 服务账号

**如果您不知道您的服务账号**，请尝试使用`gcloud`命令在执行下面的第二个单元格中获取您的服务账号。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

为Vertex AI Pipelines设置服务账号访问权限

运行以下命令，将您的服务账号授予对在上一步中创建的存储桶中读取和写入管道工件的访问权限-- 您只需要针对每个服务账号运行一次这些命令。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
from typing import NamedTuple

import kfp
from google.cloud import aiplatform
from kfp import compiler, dsl
from kfp.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,
                     OutputPath, component)

#### Vertex AI 管道常量

为 Vertex AI 管道设置以下常量：

In [ ]:
PIPELINE_ROOT = "{}/pipeline_root/shakespeare".format(BUCKET_URI)

初始化用于Python的Vertex AI SDK

为您的项目和相应的存储桶初始化用于Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

### 定义基于Python函数的pipeline组件

在本教程中，您将定义消耗参数并生成（类型化）Artifacts和参数的基于函数的组件。函数可以通过三种方式生成Artifacts：

* 使用`OutputPath`接受输出本地路径
* 接受`OutputArtifact`，该输出将函数提供给输出artifact的元数据的句柄
* 在`NamedTuple`中返回一个`Artifact`（或`Dataset`，`Model`，`Metrics`等）

这些生成Artifacts的选项将得到演示。

#### 定义预处理组件

第一个组件定义，`preprocess`，显示了一个输出两个`Dataset` Artifacts以及一个输出参数的组件。（对于此示例，数据集不反映真实数据）。

对于参数输出，您通常会使用这里所示的方法，使用`OutputPath`类型来处理“较大”数据。对于“小数据”，比如一个短字符串，可能更方便使用第二个组件中所示的`NamedTuple`函数输出。

In [ ]:
@component(base_image="python:3.9")
def preprocess(
    # An input parameter of type string.
    message: str,
    # Use Output to get a metadata-rich handle to the output artifact
    # of type `Dataset`.
    output_dataset_one: Output[Dataset],
    # A locally accessible filepath for another output artifact of type
    # `Dataset`.
    output_dataset_two_path: OutputPath("Dataset"),
    # A locally accessible filepath for an output parameter of type string.
    output_parameter_path: OutputPath(str),
):
    """'Mock' preprocessing step.
    Writes out the passed in message to the output "Dataset"s and the output message.
    """
    output_dataset_one.metadata["hello"] = "there"
    # Use OutputArtifact.path to access a local file path for writing.
    # One can also use OutputArtifact.uri to access the actual URI file path.
    with open(output_dataset_one.path, "w") as f:
        f.write(message)

    # OutputPath is used to just pass the local file path of the output artifact
    # to the function.
    with open(output_dataset_two_path, "w") as f:
        f.write(message)

    with open(output_parameter_path, "w") as f:
        f.write(message)

#### 定义训练组件

第二个组件定义，`train`，作为输入定义了一个 `Dataset` 类型的 `InputPath`，以及一个 `Dataset` 类型的 `InputArtifact`（还有其他参数输入）。它使用 `NamedTuple` 格式来输出函数。如图所示，这些输出可以是 Artifacts 也可以是参数。

此外，该组件还将一些指标元数据写入到 `model` 输出的 Artifact 中。当流水线运行时，这些信息将在云控制台用户界面上显示。

In [ ]:
@component(
    base_image="python:3.9",  # Use a different base image.
)
def train(
    # An input parameter of type string.
    message: str,
    # Use InputPath to get a locally accessible path for the input artifact
    # of type `Dataset`.
    dataset_one_path: InputPath("Dataset"),
    # Use InputArtifact to get a metadata-rich handle to the input artifact
    # of type `Dataset`.
    dataset_two: Input[Dataset],
    # Output artifact of type Model.
    imported_dataset: Input[Dataset],
    model: Output[Model],
    # An input parameter of type int with a default value.
    num_steps: int = 3,
    # Use NamedTuple to return either artifacts or parameters.
    # When returning artifacts like this, return the contents of
    # the artifact. The assumption here is that this return value
    # fits in memory.
) -> NamedTuple(
    "Outputs",
    [
        ("output_message", str),  # Return parameter.
        ("generic_artifact", Artifact),  # Return generic Artifact.
    ],
):
    """'Mock' Training step.
    Combines the contents of dataset_one and dataset_two into the
    output Model.
    Constructs a new output_message consisting of message repeated num_steps times.
    """

    # Directly access the passed in GCS URI as a local file (uses GCSFuse).
    with open(dataset_one_path) as input_file:
        dataset_one_contents = input_file.read()

    # dataset_two is an Artifact handle. Use dataset_two.path to get a
    # local file path (uses GCSFuse).
    # Alternately, use dataset_two.uri to access the GCS URI directly.
    with open(dataset_two.path) as input_file:
        dataset_two_contents = input_file.read()

    with open(model.path, "w") as f:
        f.write("My Model")

    with open(imported_dataset.path) as f:
        data = f.read()
    print("Imported Dataset:", data)

    # Use model.get() to get a Model artifact, which has a .metadata dictionary
    # to store arbitrary metadata for the output artifact. This metadata is
    # recorded in Managed Metadata and can be queried later. It also shows up
    # in the Google Cloud console.
    model.metadata["accuracy"] = 0.9
    model.metadata["framework"] = "Tensorflow"
    model.metadata["time_to_train_in_seconds"] = 257

    artifact_contents = "{}\n{}".format(dataset_one_contents, dataset_two_contents)
    output_message = " ".join([message for _ in range(num_steps)])
    return (output_message, artifact_contents)

#### 定义read_artifact_input组件

最后，您定义一个小组件，它以`train`组件函数返回的`generic_artifact`作为输入，并读取并打印Artifact的内容。

In [ ]:
@component(base_image="python:3.9")
def read_artifact_input(
    generic: Input[Artifact],
):
    with open(generic.path) as input_file:
        generic_contents = input_file.read()
        print(f"generic contents: {generic_contents}")

定义一个使用您的组件和Importer的流水线

接下来，定义一个流水线，使用在之前部分构建的组件，并展示`kfp.dsl.importer`的使用。

此示例使用`importer`从现有URI创建`Dataset` artifact。

请注意，`train_task`步骤接受`preprocess_task`步骤的三个输出作为输入，以及`importer`步骤的输出。
在"train"输入中，我们引用`preprocess`的`output_parameter`，该参数直接提供输出字符串。

`read_task`步骤接受`train_task`的`generic_artifact`输出作为输入。

In [ ]:
@dsl.pipeline(
    # Default pipeline root. You can override it when submitting the pipeline.
    pipeline_root=PIPELINE_ROOT,
    # A name for the pipeline. Use to determine the pipeline Context.
    name="metadata-pipeline-v2",
)
def pipeline(message: str):
    importer = kfp.dsl.importer(
        artifact_uri="gs://ml-pipeline-playground/shakespeare1.txt",
        artifact_class=Dataset,
        reimport=False,
    )
    preprocess_task = preprocess(message=message)
    train_task = train(
        dataset_one_path=preprocess_task.outputs["output_dataset_one"],
        dataset_two=preprocess_task.outputs["output_dataset_two_path"],
        imported_dataset=importer.output,
        message=preprocess_task.outputs["output_parameter_path"],
        num_steps=5,
    )
    read_task = read_artifact_input(  # noqa: F841
        generic=train_task.outputs["generic_artifact"]
    )

编译管道

接下来，编译管道。

In [ ]:
compiler.Compiler().compile(
    pipeline_func=pipeline, package_path="lightweight_pipeline.yaml"
)

运行管道

接下来，运行管道。

In [ ]:
DISPLAY_NAME = "shakespeare"

job = aiplatform.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="lightweight_pipeline.yaml",
    pipeline_root=PIPELINE_ROOT,
    parameter_values={"message": "Hello, World"},
    enable_caching=False,
)

job.run()

点击生成的链接以在云控制台中查看您的运行。

在Google云控制台中，许多管道DAG节点在单击时会展开或折叠。这是DAG的部分展开视图（单击图像查看更大的版本）。

<a href="https://storage.googleapis.com/amy-jo/images/mp/artifact_io2.png" target="_blank"><img src="https://storage.googleapis.com/amy-jo/images/mp/artifact_io2.png" width="95%"/></a>

删除流水线作业

您可以使用`delete()`方法来删除流水线作业。job.delete()

In [ ]:
job.delete()

清理

要清理此项目中使用的所有Google Cloud资源，可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的个别资源。

In [ ]:
delete_bucket = False

if delete_bucket:
    ! gsutil rm -r $BUCKET_URI

! rm lightweight_pipeline.yaml